In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack, hstack
import fitsio
from astropy.io import fits
import healpy as hp
from astropy import wcs

from scipy.ndimage.filters import gaussian_filter
from pathlib import Path
from multiprocessing import Pool

from scipy import stats

In [2]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [3]:
image_dir = '/global/cfs/cdirs/cosmo/staging/'
mask_dir = '/global/cfs/cdirs/desi/users/rongpu/dr9/90prime_moth_mask/20160111'

surveyccd_path = '/global/project/projectdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-90prime-dr9.fits.gz'

# image_path_list = glob.glob(os.path.join(image_dir, '*ooi*.fits.fz'))
ccd = Table(fitsio.read(surveyccd_path))

In [4]:
# mask = ccd['ccd_cuts']==0
# ccd = ccd[mask]

In [5]:
# Only keep unique exposures
_, idx = np.unique(ccd['expnum'], return_index=True)
ccd = ccd[idx]

In [6]:
ccdnum_list = [1, 2, 3, 4]
ccd_ra = [0.2813, 0.2813, -0.2813, -0.2813]
ccd_dec = [0.263, -0.263, 0.263, -0.263]

binsize = 8
pix_size = 0.454/3600*binsize
vrange = 1.

In [7]:
expnum_min = 73990064
expnum_max = 73990184

mask = (ccd['expnum']>=expnum_min) & (ccd['expnum']<=expnum_max)
print(np.sum(mask))
expnum_list = np.unique(ccd['expnum'][mask])
print(len(expnum_list))
print(expnum_list)

118
118
 expnum 
--------
73990064
73990065
73990066
73990067
73990068
73990069
73990070
73990071
73990072
73990073
     ...
73990174
73990175
73990176
73990177
73990178
73990179
73990180
73990181
73990182
73990183
73990184
Length = 118 rows


In [8]:
idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]
print(np.sum(ccd['ccd_cuts'][idx]==0))

118


In [30]:
idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/results_20160111.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()
        
        ######################## New mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(mask_dir, fn_ood)
        
        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

426 g 73990064
427 g 73990065
428 g 73990066
429 g 73990067
430 g 73990068
431 g 73990069
432 g 73990070
433 g 73990071
434 g 73990072
435 g 73990073
436 g 73990074
437 g 73990075
438 g 73990076
439 g 73990077
440 g 73990078
441 g 73990079
442 g 73990080
443 g 73990081
444 g 73990082
445 g 73990083
446 g 73990084
447 g 73990085
448 g 73990086
449 g 73990087
450 g 73990088
451 g 73990089
452 g 73990090
453 g 73990091
454 g 73990092
455 g 73990093
456 g 73990094
457 g 73990095
458 g 73990096
459 g 73990097
460 g 73990098
461 g 73990099
462 g 73990100
463 g 73990101
464 g 73990102
465 g 73990103
466 g 73990104
467 g 73990105
468 g 73990106
469 g 73990107
470 g 73990108
471 g 73990109
472 g 73990110
473 g 73990111
474 g 73990112
475 g 73990113
476 g 73990114
477 g 73990115
478 g 73990116
479 g 73990117
480 g 73990118
481 g 73990119
482 g 73990120
483 g 73990121
484 g 73990122
485 g 73990123
486 g 73990124
487 g 73990125
488 g 73990126
489 g 73990127
490 g 73990128
491 g 73990129
492 g 7399

------
## Compare old and new mask

In [35]:
idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/compare_20160111.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()
        
        ######################## compare mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path_old = os.path.join(image_dir, fn_ood)
        mask_path_new = os.path.join(mask_dir, fn_ood)
        
        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood_old = fits.getdata(mask_path_old, extname='ccd'+str(ccdnum))
            ood_new = fits.getdata(mask_path_new, extname='ccd'+str(ccdnum))
            img[ood_old!=ood_new] = 1.
            img[ood_old==ood_new] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

426 g 73990064
427 g 73990065
428 g 73990066
429 g 73990067
430 g 73990068
431 g 73990069
432 g 73990070
433 g 73990071
434 g 73990072
435 g 73990073
436 g 73990074
437 g 73990075
438 g 73990076
439 g 73990077
440 g 73990078
441 g 73990079
442 g 73990080
443 g 73990081
444 g 73990082
445 g 73990083
446 g 73990084
447 g 73990085
448 g 73990086
449 g 73990087
450 g 73990088
451 g 73990089
452 g 73990090
453 g 73990091
454 g 73990092
455 g 73990093
456 g 73990094
457 g 73990095
458 g 73990096
459 g 73990097
460 g 73990098
461 g 73990099
462 g 73990100
463 g 73990101
464 g 73990102
465 g 73990103
466 g 73990104
467 g 73990105
468 g 73990106
469 g 73990107
470 g 73990108
471 g 73990109
472 g 73990110
473 g 73990111
474 g 73990112
475 g 73990113
476 g 73990114
477 g 73990115
478 g 73990116
479 g 73990117
480 g 73990118
481 g 73990119
482 g 73990120
483 g 73990121
484 g 73990122
485 g 73990123
486 g 73990124
487 g 73990125
488 g 73990126
489 g 73990127
490 g 73990128
491 g 73990129
492 g 7399

FileNotFoundError: [Errno 2] No such file or directory: '/global/cfs/cdirs/desi/users/rongpu/dr9/90prime_moth_mask/20160111/90prime/CP/V2.3/CP20160111/ksb_160112_113411_ood_g_ls9.fits.fz'

---
## All images from CP20160111

In [9]:
mask = np.array(['CP20160111' in tmp for tmp in ccd['image_filename']])
print(np.sum(mask))
print(np.sum(ccd['ccd_cuts'][mask]==0))
idx = np.where(mask)[0]

# idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/images_20160111.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(9, 8))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}\n{}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

153
143
399 g 73990037
400 g 73990038
401 g 73990039
402 g 73990040
403 g 73990041
404 g 73990042
405 g 73990043
406 g 73990044
407 g 73990045
408 g 73990046
409 g 73990047
410 g 73990048
411 g 73990049
412 g 73990050
413 g 73990051
414 g 73990052
415 g 73990053
416 g 73990054
417 g 73990055
418 g 73990056
419 g 73990057
420 g 73990058
421 g 73990059
422 g 73990060
423 g 73990061
424 g 73990062
425 g 73990063
426 g 73990064
427 g 73990065
428 g 73990066
429 g 73990067
430 g 73990068
431 g 73990069
432 g 73990070
433 g 73990071
434 g 73990072
435 g 73990073
436 g 73990074
437 g 73990075
438 g 73990076
439 g 73990077
440 g 73990078
441 g 73990079
442 g 73990080
443 g 73990081
444 g 73990082
445 g 73990083
446 g 73990084
447 g 73990085
448 g 73990086
449 g 73990087
450 g 73990088
451 g 73990089
452 g 73990090
453 g 73990091
454 g 73990092
455 g 73990093
456 g 73990094
457 g 73990095
458 g 73990096
459 g 73990097
460 g 73990098
461 g 73990099
462 g 73990100
463 g 73990101
464 g 73990102
46